[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/http.ipynb)

Tento soubor je součástí sestavy elektronických studijních opor [Příběhy dat: Výpočetní přístupy ke studiu kultury a společnosti](https://github.com/CCS-ZCU/pribehy-dat/tree/master). 


# HTTP: Dotazování webu

### Úvod a cíle kapitoly

V této kapitole se podíváme na získání dat z různých internetových platforem.  Webový prohlížeč je jen jeden ze způsobů, jak využívat internet. Mnohé aplikace, které máme v našich počítačích i chtrých telefonech, získávají z internetu data, aniž by webový prohlížeč jakkoli vstupoval do hry. Python a Jupyter notebooky mohou být využitý jako takováto aplikace. K mnohým datům, která jsou dostupná na internetu, se tak lze dostat přímo z našeho Python prostředí, aniž bychom museli data nejprve stahovat a poté do Pythonu načítat ve formě souborů.

Tato data se z webu získávají typicky pomocí protokolu HTTP (Hypertext Transfer Protocol): Aplikace vznese dotaz v podobě konkrétní URL adresy, na který webovy server podá odpověď ve formě dat. Tento proces se vlastně děje i na pozadí našeho prohlížeče: Přes zadanou URL adresu doazujeme určitý webový server, který našemu prohlížeči jako odpověď vrátí tzv. zdrojový kód stránky ve formátu HTML. Vyzkoušejme si to nyní na několika příkladech.


Jako první příklad nám mohou posloužit webové stránky hesel ze Slovníku českých filozofů dostupná z tohoto rozcestníku: https://filozofie.phil.muni.cz/vyzkum/publikace/scf/abecedni-seznam

In [ ]:
# naimportujeme si několik knihoven
import requests # python knihovna pro vznášení HTTP dotazů
from bs4 import BeautifulSoup # python knihovna pro práci s daty ve formátu html či xml
import pandas as pd

In [ ]:
# url adresa hesla věnovaného Jiřímu Fialovi:
url = "https://www.phil.muni.cz/fil/scf/komplet/fiala.html"

In [ ]:
# na tuto webovou adresu nyní vzneseme dotaz pomocí knihovny requests
# odpověď si uložíme do proměnné `resp`:
resp = requests.get(url)

In [ ]:
# odpověď si vypíšeme
resp

Pokud máme funkční připojení k internetu a a validní URL adresu, odpověď by měla být: "<Response [200]>".
Avšak v případě, že jsme například zvolili neexistující URL, odpověď bude jiná. Vyzkoušejme:

In [ ]:
url = "https://www.phil.muni.cz/fil/scf/komplet/plato.html"
resp_test = requests.get(url)
resp_test

Vraťme se však zpět k naší odpovědi (objektu `resp`) z funkční URL adresy. Tento objekt totiž v sobě nese mnohem více, než jen informaci o tom, zda jsme obdrželi validní odpověď. Tato další data se skrývají zejména pod atributem `.text` (či `.content`)

In [ ]:
url = "https://www.phil.muni.cz/fil/scf/komplet/fiala.html"
resp = requests.get(url)
resp.text

Co zde vidíme? Jedná se o zdrojový kód příslušné webové. Přesně z těchto dat náš prohlížeč vychází, když nám prezentuje vybranou webovou stránku. V tomto kroku je daný kód zapouzdřený jako proměnná typu string. Všimněme si, že zde máme problém se znakovou sadou: např. "pøekladù" namísto "překladů". Tento nedostatek však rychle napravíme a na text se podíváma ještě jednou: 


In [ ]:
resp.encoding = 'windows-1250' # volba znakové sady
resp.text

Nyní je již vše správně. Tato data v sobě však nesou více než prostý text - jsou ve formátu HTML, sestávající z mnoha tagů, které určují kde začíná a končí řádka, vyznačují další hypertextové odkazy apod. Nyní proto použijeme knihovny `BeautifulSoup`, abychom tento text interpretovali (parsovali) jako HTML kód, ve kterém se lze pohybovat jako v jakémsi stromu:

In [ ]:
soup = BeautifulSoup(resp.text, "html.parser")

Objekt soup v sobě skrývá navigovatelnou stromovou strukturu celé html stránky. Můžeme zde nyní přistupovat k datům z jednotlivých tagů případně je i měnit. Názorné příklady jsou k dispozici v dokumentaci knihovny v sekci "Quick start". 

Například pod tagem `title` se skrývá název příslušné webové stránky:

In [ ]:
print(soup.prettify())

In [ ]:
soup.title

Pokud nás zajímá pouze textový obsah daného tagu, příkaz ještě doplníme o atribut `string` nebo metodu `get_text()`

In [ ]:
soup.title.string

Po troše experimentování a prozkoumávání můžeme například zjistit, že datum narození a úmrtí se v dané struktuře nachází zde:

In [ ]:
soup.body.find_all("p")[2].find_all("strong")[0].string

In [ ]:
soup.body.find_all("p")[2].find_all("strong")[1].string


To jsou velice zajímavá a cenná metadata. Je stejná struktura zachována i na dalších stránkách ve slovníku? To nyní prověříme.

In [ ]:
url = "https://www.phil.muni.cz/fil/scf/komplet/machjs.html"
resp = requests.get(url)
resp.encoding = 'windows-1250'
soup = BeautifulSoup(resp.text, "html.parser")
soup.title.string # vypíšeme název html

In [ ]:
soup.body.find_all("p")[2].find_all("strong")[0].get_text() # pokusíme se nalézt datum a místo narození

In [ ]:
soup.body.find_all("p")[2].find_all("strong")[1].get_text() # pokusíme se nalézt datum a místo úmrtí

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
exec(requests.get("https://sciencedata.dk/shared/856b0a7402aa7c7258186a8bdb329bd3?download").text)
kontrola_pruchodu(ntb="http", arg1=url)

Práce s mnoha webovými stránkami najednou

Webové stránky Slovníku českých filozofů mají abecední rozcestník: https://filozofie.phil.muni.cz/vyzkum/publikace/scf/abecedni-seznam. Při ohledání zdrojového kódu této stránky se ukazuje, že obsahuje URL adresy všech hesel ve slovníku. Srdce datového analytika zaplesá: To znamená, že můžeme jednu po druhé procházet všechny dané webové stránky a získat z nich data, která nás zajímají, například informace o datech a místech narození a úmrtí všech zahrnutých osobností. 

Nejprve budeme postupovat stejně jako výše: Pomocí `requests` získáme objekt `resp`, jehož datový obsah v podobě zdrojového kódu příslušné webové adresy naparsujeme pomocí `BeautifulSoup()` do navigovatelného stromu. 

In [ ]:
url = "https://filozofie.phil.muni.cz/vyzkum/publikace/scf/abecedni-seznam"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

Nyní pomocí cyklu FOR projedeme všechny tagy `a` (stanardní tag pro hypertextový odkaz). Pokud se v hodnotě atributu "href" tohoto tagu nachází URL adresa začínající stejně jako URL adresy jednotlivých hesel ve slovníků, extrahujeme tuto adresu a přidáme ji na seznam `filtered_hrefs`. (Ve zdrojovém kódu se může vyskytovat mnoho dalších URL adres, odkazujících např. kamsi na web Masarykovy univerzity, ty nás ale nyní nezajímají, proto tato podmínka.)

In [ ]:
filtered_hrefs = []

for a_tag in soup.find_all('a'):
    href = a_tag.get('href')    # Extract 'href' attribute value.
    if href and href.startswith('http://www.phil.muni.cz/fil/scf/komplet/'):   # condition
        filtered_hrefs.append(href)

vypišme si prvních 10 URL adres:

In [ ]:
filtered_hrefs[:10]

Nyní vytvoříme cyklus FOR, v rámci kterého získáme zdrojový kód HTML stránky každé z těchto URL adres pomocí `requests`, zpracujeme jej pomocí `BeautifulSoup` a pokusíme se získat data se jmény jednotlivých filozofů a o místech a datech narození a úmrtí. 

In [ ]:
failed = []
slovnik_data = []
for url in filtered_hrefs[:10]:
    try:
        resp = requests.get(url)
        resp.encoding = 'windows-1250'
        soup = BeautifulSoup(resp.text, "html.parser")
        title = soup.title.string
        birth = soup.body.find_all("p")[2].find_all("strong")[0].string
        try:
            death = soup.body.find_all("p")[2].find_all("strong")[1].string
        except:
            death = None
        if "*" not in str(birth):
            birth = None
        if "†" not in str(death):
            death = None
        slovnik_data.append({"url" : url, "name" : title, "birth" : birth, "death" : death})
    except:
        failed.append(url)

Z dat, která jsme takto získali, vytvoříme velice snadno tabulku, respektivě objekt typu `pandas.DataFrame`:

In [ ]:
slovnik_df = pd.DataFrame(slovnik_data) # vytvoříme dataframe
slovnik_df = slovnik_df.replace(to_replace=r'\r\n', value='', regex=True) # drobné čičtění 
slovnik_df # dataframe si vypíšeme

Vidíme relativně pěkně naformátovanou tabulku se čtyřmi sloupci. Některé hodnoty jsou prázdné (`None`), například ve sloupci "death" v případě, že se jedná o žijícího autora. V jiných případech se zdá, že některá hesla jsou naformátovaná odlišně.

 Z praktických důvodů jsme se však omezili pouze na 10 prvních hesel. Dropnou úpravou (odstraněním "[:10"]) však lze skript snadno přenastavit tak, aby se aplikoval na všechny URL adresy na seznamu. Tomu se však nyní společně vyhneme, abychom příslušnou webovou doménu zbytečně nezavalovali tisíci HTTP dotazy. Na místo toho si tato kompletní data načteme z místa, kam jsem je dopředu uložil a vypíšeme si první 20 položek:

In [ ]:
slovnik_df.to_csv("../data/slovnik_df.csv", index=False)

In [ ]:
slovnik_df = pd.read_csv("https://raw.githubusercontent.com/CCS-ZCU/pribehy-dat/master/data/slovnik_df.csv")
slovnik_df.head(20)

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
kontrola_pruchodu(ntb="http", arg1="http2")

V některé z dalších kapitol se k těmto datům vrátíme a budeme pracovat se sloupci "birth" a "death", které obsahují cenné časoprostorové informace. Tyto informace nyní nejsou v podobě vhodné pro kvantitativní analýzu. My si však ukážeme, jak je přetavit do podoby, aby pro tuto analýzu byla tato dato vhodná.  

Je nasnadě, že podobný přístup lze aplikovat na mnohé webové stránky, a to zvláště tehdy, kdy se jedná o velké množství podobně naformátovaných stránek. Často to je jediný přístup, jak se k podobným datům dostat. Této metodě se také říká "web scraping" a je někdy problematická z etického hlediska. Některé webové stránky výslovně zakazují, aby data, která se na nich nacházejí, byla dolována podobným způsobem. Někdy je to motivováno obavou z přetížení příslušného webového serveru mnoha dotazy, jindy spíše s ohledem na autorství. Často se však zdá, že tvůrce daných webových stránek vůbec nenapadlo, že by jejich dat někdo mohl chtít využívat tímto způsobem.